In [1]:
import glob
import os, re
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [2]:
#     1. 소문자로 바꾸고, 양쪽 공백을 지우기
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꾼다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다
#     5. 다시 양쪽 공백을 지운다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가한다
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [3]:
# 정제된 문장 모으기
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(sentence) >= 15: continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

In [4]:
# Tokenizer, pad_sequence
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들기
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꾸기
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성하기
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰준다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰준다
    # (문장 앞에 패딩을 붙이고 싶다면, padding='pre'를 사용)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 143   3 ...   0   0   0]
 [  2 143   3 ...   0   0   0]
 [  2  55 912 ...   0   0   0]
 ...
 [  2 193 571 ...   0   0   0]
 [  2  62  22 ...   0   0   0]
 [  2  32  16 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fda77401c70>


In [5]:
# tensor에서 마지막 토큰(<pad>일 확률이 높음)을 잘라내서 소스 문장을 생성
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2 143   3   0   0   0   0   0   0   0   0   0]
[143   3   0   0   0   0   0   0   0   0   0   0]


In [20]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                    tgt_input, 
                                                    test_size=0.2, 
                                                    random_state=7)

print("Source Train: ", enc_train.shape)
print("Target Train: ", dec_train.shape)
print("Sourt Val: ", enc_val.shape)

Source Train:  (9732, 12)
Target Train:  (9732, 12)
Sourt Val:  (2434, 12)


# 훈련데이터셋

In [11]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 12), (256, 12)), types: (tf.int32, tf.int32)>

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 3072
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=5, validation_data=(enc_val, dec_val))

Epoch 1/5
305/305 [==============================] - 22s 58ms/step - loss: 1.5855 - val_loss: 1.3142
Epoch 2/5
305/305 [==============================] - 17s 57ms/step - loss: 1.2369 - val_loss: 1.2451
Epoch 3/5
305/305 [==============================] - 17s 57ms/step - loss: 1.1358 - val_loss: 1.1850
Epoch 4/5
305/305 [==============================] - 17s 56ms/step - loss: 1.0446 - val_loss: 1.1373
Epoch 5/5
305/305 [==============================] - 17s 56ms/step - loss: 0.9681 - val_loss: 1.1058


# 테스트 데이터셋

In [21]:
BUFFER_SIZE = len(enc_val)
BATCH_SIZE = 256
steps_per_epoch = len(enc_val) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

test_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val))
test_dataset = test_dataset.shuffle(BUFFER_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
test_dataset

<BatchDataset shapes: ((256, 12), (256, 12)), types: (tf.int32, tf.int32)>

In [22]:
results = model.evaluate(test_dataset)

9/9 [==============================] - 1s 74ms/step - loss: 1.1060


In [23]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만든다
    #    1. 입력받은 문장의 텐서를 입력
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑는다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙인다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마친다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [28]:
generate_text(model, tokenizer, init_sentence="<start> i love ", max_len=20)

'<start> i love you <end> '

# 회고

- **어려웠던 점**: 코드를 하나씩 분석했는데 자연어처리는 아직 어렵게 다가오는 것 같다. 모든 코드를 이해한 것은 아니지만 한 다섯번 훑어보니까 조금은 이해된 것 같다.
- 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**: generate_text 함수에서 while문 돌리는 걸 아직 다 이해하지 못했다.
- 루브릭 평가 지표를 맞추기 위해 **시도한 것들**: embedding_size, hidden_size를 올려보았는데 도움이 되었는지는 잘 모르겠다.
- **자기 다짐**: 위에 코드와 함께 적어놓은 설명들을 찬찬히 읽고 전반적인 흐름을 머릿속에 저장해 놓자.